# Práctica 2. Sistemas de Ecuaciones Diferenciales

## 1. Resolución analítica


En la práctica anterior, vimos cómo usar el comando `dsolve` (dentro del paquete `sympy`) para resolver ecuaciones diferenciales ordinarias. Cabe recordar que esta instrucción nos devuelve la solución analítica, **cuando esta existe**. Como vemos a continuación, este comando también sirve para resolver sistemas de ecuaciones diferenciales. Consideremos el siguiente sistema, correspondiente al ejercicio 1(g) del Tema 3.


$$ \begin{cases}x'+2+2y'=4x+5y \\ 2x'-y'=3x \end{cases}, \qquad x(0)=1, y(0)=-1. $$

In [ ]:
import sympy as sym   # por comodidad, llamaremos al paquete sympy como 'sym'

t=sym.symbols('t'); 
x=sym.symbols('x' , cls = sym.Function)
y=sym.symbols('y' , cls = sym.Function)

# introducimos las ecuaciones entre corchetes y separadas por comas
# las introducimos pasándolo todo al lado izquierdo, como si estuviesen igualadas a cero
# después las funciones-incógnitas (aunque se pueden omitir, pues normalmente Python las detecta automáticamente)
# las condiciones iniciales se introducen a través de "ics", entre llaves, y con dos puntos (es decir, no ponemos x(0)=1)

sol = sym.dsolve([x(t).diff(t)+2+2*y(t).diff(t)-4*x(t)-5*y(t), 2*x(t).diff(t)-y(t).diff(t)-3*x(t)], 
             [x(t), y(t)],
             ics={x(0):1, y(0):-1})

Python nos devuelve un vector con dos componentes. Podemos acceder a ellas, de una manera más elegante, a través del comando `display`.

In [ ]:
display(sol[0])
display(sol[1])

También podemos sustituir las soluciones en cierto valor a través de la instrucción `.subs()`.

In [ ]:
sol[0].subs(t, 0)

In [ ]:
sol[1].subs(t, 0)

## 2. Resolución numérica 


Existen sistemas de ecuaciones diferenciales que no admiten soluciones analíticas, como el siguiente. 


$$ \begin{cases} x'=1-y \\ y'=x^2-y^2  \end{cases}$$


En tales casos, existen **métodos numéricos** para aproximar las soluciones. En esta práctica, utilizaremos la instrucción `odeint` del paquete `scipy.integrate`. No profundizaremos en esta práctica sobre el método de aproximación numérica que utiliza dicho comando, ni sobre otros métodos numéricos existentes. Para ejecutar dicha instrucción, deberemos introducir:
- Una función que describa el sistema de ecuaciones diferenciales.
- Un corchete con las condiciones iniciales.
- Una sucesión de tiempos (creciente) en los cuales se calculará la solución.
- Solo si es necesario, un vector de parámetros, en forma de tupla. 

Trabajemos, por ejemplo, con las condiciones iniciales $x(0)=-1.5$ e $y(0)=0.5$, y con cien puntos equiespaciados en el intervalo $[0,3]$.

In [ ]:
import numpy as np
from scipy.integrate import odeint


def sistema(variables,t):          # función que nos devuelve las ecuaciones del sistema; en este caso se podría obviar el parámetro t
    [x,y] = variables
    dxdt = 1-y
    dydt = x**2-y**2
    return [dxdt,dydt]

z0 = [-1.5,0.5]                    # condiciones iniciales (NO introducir como 'array')
t = np.linspace(0,3,100)           # vector representando una sucesión de tiempos

sol = odeint(sistema,z0,t)         # no hace falta en este caso concreto introducir el vector de parámetros en 'odeint'
print(sol[0:3,:])

A continuación, realizamos una representación de las soluciones, ambas en función del tiempo. Para ello, debemos primeramente cargar el paquete `matplotlib.pyplot` para poder realizar gráficos en Python. El resto de instrucciones aparecen explicadas en comentarios.

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(7, 5))                                                 # forzamos que el tamaño de la imagen sea 7x5 pulgadas
                                                                           # SIEMPRE debe aparecer lo PRIMERO
plt.xlabel("tiempo t", fontsize=15)                                        # nombre del eje OX y tamaño de letra
plt.title("Gráfica de las soluciones", fontweight="bold", fontsize=15)     # nombre del gráfico, en negrita, y tamaño de letra
plt.plot(t,sol[:,0],'b',label='x(t)')                                      # dibuja x(t), para cada instante t, en azul, y le asociamos el nombre 'x(t)'
plt.plot(t,sol[:,1],'r',label='y(t)')                                      # dibuja y(t), para cada instante t, en rojo, y le asociamos el nombre 'y(t)'
plt.legend(fontsize=15)                                                    # muestra en el gráfico las leyendas, con los nombres anteriormente mencionados y tamaño 15 de letra
plt.show()                                                                 # muestra el gráfico

Si ahora representamos $y$ en función de $x$, es decir, si obviamos $t$, entonces podemos ver el **diagrama de fase** asociado a esta **condición inicial concreta**.

In [ ]:
plt.xlabel("x(t)", fontsize=15)     
plt.ylabel("y(t)", fontsize=15)                                   
plt.title("Diagrama de fase para (x(0),y(0))=(-1.5,0.5)", fontweight="bold", fontsize=15)  
plt.plot(sol[:,0],sol[:,1],'b')  
plt.plot(-1.5,0.5,'o',markersize=8)                          # dibujamos la situación inicial, en este caso (-1.5, 0.5)                                                                                                       
plt.show()                                                              

## 3. Diagramas de fase

### 3.1 Sistemas de ecuaciones diferenciales lineales

Los sistemas de ecuaciones diferenciales **lineales** y autónomos $2\times 2$ admiten una representación matricial de la forma $\begin{pmatrix} x'(t) \\ y'(t) \end{pmatrix} = A \begin{pmatrix} x(t)\\y(t)\end{pmatrix}$. 


Como sabemos, en estos casos podemos representar diagramas de fase en función de los autovalores y los autovectores de $A$. Sea $\sigma(A)=\{\lambda_1, \lambda_2\}$. Tal y como vimos en teoría, no vamos a trabajar con el caso donde $\lambda_i=0$ para algún $i\in\{1,2\}$. Por tanto, $|A|=\lambda_1 \cdot \lambda_2 \neq 0$ y $A$ siempre será invertible, con único punto de equilibrio $(0,0)$. 


En la práctica `L3. Cadenas de Markov` de la asignatura Álgebra vimos cómo obtener dichos autovalores y autovectores. Lo recordamos a continuación. Trabajaremos con el sistema siguiente. $$ \begin{cases}x'=x+4y \\ y'=2x+3y\end{cases}$$



In [ ]:
from scipy import linalg as la

A=np.array([[1,4],[2,3]])
autovalores, autovectores = la.eig(A)
print(autovalores)
print(autovectores)

Vemos que $\sigma(A)=\{-1,5\}$, luego $|A|=-5<0$ y por tanto tenemos que el punto de equilibrio $(0,0)$ es un **punto de silla**. A continuación, representamos el diagrama de fase.

In [ ]:
plt.figure(figsize=(9, 7))

plt.xlim(-5, 5)                                # límites para el gráfico en el eje OX
plt.ylim(-5, 5)                                # límites para el gráfico en el eje OY

plt.axline([0,0], autovectores[:,0], linewidth=2, color='r')     # recta que pasa por el (0,0), en dirección del primer autovector
plt.axline([0,0], autovectores[:,1], linewidth=2, color='r')     # recta que pasa por el (0,0), en dirección del segundo autovector
plt.plot(0, 0, 'ko', markersize=8)                                


def sistema(variables,t):                        
    [x,y] = variables
    dxdt = x+4*y
    dydt = 2*x+3*y
    return [dxdt, dydt]


x = np.linspace(-5, 5, 25)                      # puntos equiespaciados necesarios para construir la rejilla
y = np.linspace(-5, 5, 25)


X,Y=np.meshgrid(x, y)                           # creamos una rejilla con los puntos anteriores
DX,DY=sistema([X, Y], 0)                        # a partir de la rejilla anterior, calculamos los nuevos puntos que proporciona el sistema
                                                # no necesitamos sucesión de tiempos t, luego ponemos un valor "auxiliar" 0
M=np.sqrt(DX**2+DY**2)                          # calculamos la norma de los nuevos puntos (vectores)
M[M==0]=1                                       # si un vector es nulo (luego su norma será nula), diviremos por 1
NX=DX/M                                         # dividimos por la norma para normalizar los vectores (flechas)
NY=DY/M                                         


plt.quiver(X, Y, NX, NY, M)                     # dibujamos un mapa, donde en cada punto de la rejilla aparezca el vector (normalizado) que proporciona el sistema


plt.xlabel("x(t)", fontsize=15)                                    
plt.ylabel("y(t)",fontsize=15)
plt.title("Diagrama de fases", fontweight="bold", fontsize=15)     
plt.show()

Podemos añadirle al diagrama de fases anterior una trayectoria concreta, por ejemplo la que tiene como situación inicial $(-2,2)$. Para ello, necesitamos primero la solución que parte de ese punto.

In [ ]:
t=np.linspace(0,1,50)
z0=[-2,2]
sol=odeint(sistema,z0,t)

plt.figure(figsize=(9, 7))

plt.xlim(-5, 5)                                
plt.ylim(-5, 5) 

plt.axline([0,0], autovectores[:,0], linewidth=2, color='r')     
plt.axline([0,0], autovectores[:,1], linewidth=2, color='r')     
plt.plot(0, 0, 'ko', markersize=8)            

plt.plot(-2,2,'bo',markersize=8)
plt.plot(sol[:,0], sol[:,1],'b',label='trayectoria')

plt.quiver(X, Y, NX, NY, M)                     
plt.xlabel("x(t)", fontsize=15)                                    
plt.ylabel("y(t)",fontsize=15)
plt.title("Diagrama de fases", fontweight="bold", fontsize=15)    
plt.legend(fontsize=15) 
plt.show()

A continuación, realizamos el diagrama de fases correspondiente al sistema de ecuaciones diferenciales $\begin{cases} x'=3x-5y\\ y'=x-y\end{cases}$.

In [ ]:
A=np.array([[3,-5],[1,-1]])
autovalores, autovectores = la.eig(A)
print(autovalores)

Como vemos, los autovalores son complejos con parte real positiva, por lo que sabemos que el diagrama de fases serán espirales con punto de equilibrio $(0,0)$ inestable.

In [ ]:
plt.figure(figsize=(9, 7))

plt.xlim(-5, 5)                                
plt.ylim(-5, 5)                                
plt.plot(0, 0, 'ko', markersize=8)                                


def sistema(variables, t):                        
    [x,y] = variables
    dxdt = 3*x-5*y
    dydt = x-y
    return [dxdt, dydt]


x = np.linspace(-5, 5, 25)                      
y = np.linspace(-5, 5, 25)


X,Y=np.meshgrid(x, y)                           
DX,DY=sistema([X, Y],0)                           
M=np.sqrt(DX**2+DY**2)                          
M[M==0]=1                                       
NX=DX/M                                         
NY=DY/M                                         


plt.quiver(X, Y, NX, NY, M)       

t=np.linspace(0,5,50)
sol=odeint(sistema,[-0.5,0],t)                                  # representamos también una trayectoria al azar, por ejemplo la que tiene como situación inicial (-2,2)

plt.plot(-0.5,0,'bo',markersize=8)
plt.plot(sol[:,0], sol[:,1],'b',label='trayectoria')

plt.xlabel("x(t)", fontsize=15)                                    
plt.ylabel("y(t)",fontsize=15)
plt.title("Diagrama de fases", fontweight="bold", fontsize=15)   
plt.legend(fontsize=15)  
plt.show()

### 3.2 Sistemas de ecuaciones diferenciales no lineales


Como sabemos, los sistemas de ecuaciones diferenciales **no lineales** pueden tener varios puntos de equilibrio, y en cada uno de ellos podíamos analizar la estabilidad a través del sistema linealizado. A continuación, veremos cómo hallar el diagrama de fases del siguiente sistema de ecuaciones lineales que aparece en la sección 2.


$$ \begin{cases} x'=1-y \\ y'=x^2-y^2  \end{cases}$$


Seguiremos el mismo procedimiento anteriormente descrito. Observemos que los puntos de equilibrio se pueden hallar a través de la instrucción `sym.solve`, donde el primer parámetro son las ecuaciones (entre corchetes o paréntesis) a resolver introducidas a través de `sym.Eq` y el segundo parámetro son las variables para las cuales queremos hallar sus valores (entre corchetes o paréntesis también).

In [ ]:
# volvemos a cargar 'x' e 'y' como símbolos, pues en el último gráfico eran puntos de la rejilla
# las introducimos como variables, pues queremos hallar sus valores, NO como funciones de 't'
# solo buscamos soluciones REALES

x = sym.symbols('x', real=True)      
y = sym.symbols('y', real=True)      
ptos_equilibrio = sym.solve([sym.Eq(1-y,0), sym.Eq(x**2-y**2, 0)], [x, y])
print(ptos_equilibrio)

Ahora, ya podemos repetir las instrucciones de la sección anterior para realizar el diagrama de fases de este sistema.

In [ ]:
plt.figure(figsize=(9, 7))

plt.xlim(-5, 5)                                
plt.ylim(-5, 5)                               

plt.plot(ptos_equilibrio[0][0], ptos_equilibrio[0][1], 'ko', markersize=8) 
plt.plot(ptos_equilibrio[1][0], ptos_equilibrio[1][1], 'ko', markersize=8) 


def sistema(variables,t):                        
    [x,y] = variables
    dxdt = 1-y
    dydt = x**2-y**2
    return [dxdt, dydt]


x = np.linspace(-5, 5, 30)                      
y = np.linspace(-5, 5, 30)


X,Y=np.meshgrid(x, y)                           
DX,DY=sistema([X, Y], 0)                        
                                                
M=np.sqrt(DX**2+DY**2)                          
M[M==0]=1                                       
NX=DX/M                                         
NY=DY/M                                         


plt.quiver(X, Y, NX, NY, M)                     


plt.xlabel("x(t)", fontsize=15)                                    
plt.ylabel("y(t)",fontsize=15)
plt.title("Diagrama de fases", fontweight="bold", fontsize=15)     
plt.show()

Podemos también añadirle algunas trayectorias al diagrama anterior. Para ello, calculamos primero las soluciones que tienen como condiciones iniciales a los puntos $(0, \, 4.3)$, $(-1, \, 4)$, $(-4, \, -3.4)$, $(2, \, -2)$ y $(-1.8, -0.8)$.

In [ ]:
t1=np.linspace(0,6,50)
sol1=odeint(sistema,[0,4.3],t1)

t2=np.linspace(0,3,50)
sol2=odeint(sistema,[-1,4],t2)

t3=np.linspace(0,1.5,50)
sol3=odeint(sistema,[-4,-3.4],t3)

t4=np.linspace(0,5,50)
sol4=odeint(sistema,[2,-2],t4)

t5=np.linspace(0,3,50)
sol5=odeint(sistema,[-1.8,-0.8],t5)

Ahora, repetimos el dibujo anterior, pero añadiéndole las trayectorias de estas soluciones.

In [ ]:
plt.figure(figsize=(10, 8))

plt.xlim(-5, 5)                                
plt.ylim(-5, 5)                               

plt.plot(ptos_equilibrio[0][0], ptos_equilibrio[0][1], 'ko', markersize=8) 
plt.plot(ptos_equilibrio[1][0], ptos_equilibrio[1][1], 'ko', markersize=8) 


def sistema(variables,t):                        
    [x,y] = variables
    dxdt = 1-y
    dydt = x**2-y**2
    return [dxdt, dydt]


plt.plot(0,4.3,'go',markersize=5)
plt.plot(sol1[:,0], sol1[:,1],'g',label='sol. 1')

plt.plot(-1,4,'ro',markersize=5)
plt.plot(sol2[:,0], sol2[:,1],'r',label='sol. 2')

plt.plot(-4,-3.4,'mo',markersize=5)
plt.plot(sol3[:,0], sol3[:,1],'m',label='sol. 3')

plt.plot(2,-2,'yo',markersize=5)
plt.plot(sol4[:,0], sol4[:,1],'y',label='sol. 4')

plt.plot(-1.8,-0.8,'co',markersize=5)
plt.plot(sol5[:,0], sol5[:,1],'c',label='sol. 5')

plt.legend(fontsize=13)


x = np.linspace(-5, 5, 30)                      
y = np.linspace(-5, 5, 30)


X,Y=np.meshgrid(x, y)                           
DX,DY=sistema([X, Y], 0)                        
                                                
M=np.sqrt(DX**2+DY**2)                          
M[M==0]=1                                       
NX=DX/M                                         
NY=DY/M                                         


plt.quiver(X, Y, NX, NY, M)                     


plt.xlabel("x(t)", fontsize=15)                                    
plt.ylabel("y(t)",fontsize=15)
plt.title("Diagrama de fases", fontweight="bold", fontsize=15)     
plt.show()

## 4. Modelo SIR


Tal y como vimos en teoría, el modelo SIR sirve para estudiar la **propagación de enfermedades contagiosas y rumores**. Recordemos que si $S(t)$, $I(t)$ y $R(t)$ miden el número de susceptibles, infectados y recuperados de una cierta enfermedad/rumor en un instante concreto de tiempo $t$, entonces el modelo SIR viene determinado por el sistema de ecuaciones diferenciales 


$$ \begin{cases} 
S'= \mu N-\beta S I - \mu S \\
I'=\beta S I - \mu I - \gamma I \\
R'=\gamma I - \mu R
\end{cases},  $$ 


donde $N=S(t)+I(t)+R(t)$ es el tamaño de población (y es constante), $\beta$ es el coeficiente de transmisión, $\gamma$ es la tasa de recuperación y $\mu$ es la tasa de natalidad y coincide con la mortalidad total. 



Este modelo carece de solución analítica, pero podemos **aproximar sus soluciones numéricamente**.

In [ ]:
def sir(variables,t,N,beta,mu,gamma):
    [S,I,R] = variables
    dSdt = mu*N-beta*S*I-mu*S
    dIdt = beta*S*I-mu*I-gamma*I
    dRdt = gamma*I-mu*R
    return [dSdt, dIdt, dRdt]

Vimos en clase de teoría que si $R_0=\dfrac{\beta N}{\mu+\gamma}<1$, entonces cuando $t \rightarrow \infty$ la solución $(S(t), \, I(t))$ de las dos primeras ecuaciones tendía al punto de equilibrio $(N, \, 0)$ y por tanto la enfermedad se extinguía. Comprobémoslo con unos datos concretos, por ejemplo con $(N, \, \beta, \, \mu, \, \gamma) = (1, \, 0.1, \, 0.01, \, 0.1)$. Podemos suponer que $N$ se mide en millones de habitantes, luego $N=1$ significa un millón de habitantes. Además, supongamos que la situación inicial es $(S(0), \, I(0), \, R(0)) = (0.1, \, 0.9, \, 0)$, i.e. que están infectados 900 mil habitantes.

In [ ]:
N = 1                
beta = 0.1
mu = 0.01
gamma = 0.1

parametros = (N, beta, mu, gamma)   # debe estar en forma de tupla, tal y como vimos en la sección 2

R0 = beta*N/(mu+gamma)
print(R0)                           # podemos comprobar que R0 < 1 en este caso

sit_inicial = [0.1, 0.9, 0]

Calculemos la solución numérica para esos datos concretos y posteriormente realizamos un gráfico. En esta ocasión, **no realizaremos un diagrama de fases**, sino un gráfico donde podamos ver cómo se comportan las soluciones $S(t), I(t), R(t)$ en función del tiempo (en días). Veamos qué sucede desde $t=0$ hasta $t=450$ días.

In [ ]:
t = np.linspace(0,450,1000)
sol = odeint(sir,sit_inicial,t,parametros)

plt.figure(figsize=(10, 8))

plt.plot(t, sol[:,0], label='S(t)')
plt.plot(t, sol[:,1], label='I(t)')
plt.plot(t, sol[:,2], label='R(t)')

plt.plot(t,N*np.ones(t.shape),label='equilibrio S')
plt.plot(t,0*np.ones(t.shape),label='equilibrio I')

plt.legend(fontsize=13)

plt.xlabel("t", fontsize=17, fontweight="bold")                                    
plt.ylabel("SIR",fontsize=17, fontweight="bold")
plt.show()

Si $R_0=\dfrac{\beta N}{\mu+\gamma}>1$, entonces cuando $t \rightarrow \infty$ la solución $(S(t), \, I(t))$ de las dos primeras ecuaciones tendía al punto de equilibrio

$$\left(\dfrac{\mu + \gamma}{\beta}, \: \dfrac{\mu}{\beta}\left(\dfrac{N\beta}{\mu+\gamma}-1\right)\right)$$ 

y por tanto la enfermedad se volvía endémica. Comprobémoslo con los datos $(N, \, \beta, \, \mu, \, \gamma) = (1, \, 0.5, \, 0.01, \, 0.1)$ y con la situación inicial $(0.9, \, 0.1, \, 0)$. Veamos qué sucede después de 250 días.

In [ ]:
N = 1                
beta = 0.5
mu = 0.01
gamma = 0.1

parametros = (N, beta, mu, gamma)   # debe estar en forma de tupla, tal y como vimos en la sección 2

R0 = beta*N/(mu+gamma)
print(R0)                           # podemos comprobar que R0 > 1 en este caso

sit_inicial = [0.9, 0.1, 0]

In [ ]:
t = np.linspace(0,250,1000)
sol = odeint(sir,sit_inicial,t,parametros)

plt.figure(figsize=(10, 8))

plt.plot(t, sol[:,0], label='S(t)')
plt.plot(t, sol[:,1], label='I(t)')
plt.plot(t, sol[:,2], label='R(t)')

plt.plot(t, ((mu+gamma)/beta)*np.ones(t.shape),label='equilibrio S')
plt.plot(t, (mu/beta)*((N*beta/(mu+gamma))-1)*np.ones(t.shape),label='equilibrio I')

plt.legend(fontsize=13)

plt.xlabel("t", fontsize=17, fontweight="bold")                                    
plt.ylabel("SIR",fontsize=17, fontweight="bold")
plt.show()

## 5. Ejercicios



**1.** Calcula la solución **analítica** (i.e. exacta) de los siguientes sistemas de ecuaciones diferenciales.

$$\text{(a)}\: \begin{cases} x''+y'-x+y=-1\\x'+y'-x=t^2 \end{cases} \qquad \qquad \qquad \text{(b)}\: \begin{cases} x'=x+2y-z \\ y'=x+z \\ z'=4x-4y+5z \end{cases}$$

**Solución:** (a) $x(t)=−C_1te^t-\frac{C_2e^{-t}}{2}-t^2-4t+(C_1-C_3)e^t-6$, $\quad y(t)=C_1e^t+C_2e^{-t}-t^2-2t-3$.

(b) $x(t)=-\frac{C_1e^t}{2}-\frac{C_2e^{2t}}{2}-\frac{C_3e^{3t}}{4}$, $\quad y(t)=\frac{C_1e^t}{2}+\frac{C_2e^{2t}}{4}+\frac{C_3e^{3t}}{4}$, $\quad z(t)=C_1e^t+C_2e^{2t}+C_3e^{3t}$.


**2.** Determina los diagramas de fase de los siguientes sistemas de ecuaciones diferenciales **lineales**. Los diagramas tendrán $x,y\in [-5,5]$ y deben ilustrar el punto de equilibrio, las rectas invariantes (en caso de autovalores reales), y la trayectoria de la solución cuando la condición inicial es $(1, -3)$.

$$\text{(a)}\: \begin{cases} x'=-2x+y \\ y'=x-2y \end{cases} \qquad \qquad \text{(b)}\: \begin{cases} x'=-x-y \\ y'=2x+y \end{cases} \qquad \qquad \text{(c)}\: \begin{cases} x'=-2x-y \\ y'=3x+2y \end{cases}$$


**Solución:** hechos en clase.


**3.** Determina los diagramas de fase de los siguientes sistemas de ecuaciones diferenciales **no lineales**. Los diagramas tendrán $x,y\in [-5,5]$ (excepto el último, que tendrá $x,y\in[0,5]$) y deben ilustrar los puntos de equilibrio y las trayectorias de dos soluciones con condiciones iniciales elegidas por ti.

$$\text{(a)}\:  \begin{cases} x'=y^3-x^3 \\ y'=1-y^3 \end{cases} \qquad \qquad \text{(b)}\: \begin{cases} x'=y-y^3 \\ y'=-x-y^2 \end{cases} \qquad \qquad\text{(c)}\:  \begin{cases} x'=x-2y\sqrt{x} \\ y'=-2y+y\sqrt{x} \end{cases} $$


**Solución:** (a) Espirales con sentido horario, punto de equilibrio $(1,1)$ asintóticamente estable. (b) Circunferencias/elipses en sentido horario con punto de equilibrio $(0,0)$ estable. Puntos de silla en $(-1, -1)$ y $(-1, 1)$, inestables. (c) Espirales con sentido antihorario, punto de equilibrio $(4,1)$ asintóticamente inestable.


**4.** En el ejercicio 12 de la lista de problemas del Tema 3 se pide plantear el sistema de ecuaciones diferenciales que modelan la viralización de un meme. Supongamos que el tiempo $t$ se mide en horas, que la tasa de hartazgo es 0.2 y que la tasa de volver a memetizarse es 0.1. Supongamos también que actualmente hay un $10\%$ de memetizados por un meme reciente, luego no hay nadie harto aún. Realiza un gráfico donde se muestre cómo evolucionan $S(t), M(t)$ y $H(t)$ durante las primeras 100 horas.
    - Al final de dicho período, ¿hay más personas hartas del meme o memetizadas?
    - Si se define un meme viral como aquel que provoca un porcentaje de memetizados superior al $50\%$ de la población total en algún instante $t$, ¿se puede concluir que el meme fue viral?
    

**Solución:** hay más personas hartas del meme. Y no fue viral el meme, pues el porcentaje de memetizados nunca superó ni siquiera el $30\%$.